In [1]:
import gym
import random

In [2]:
BOARD_SIZE = 8
EPSILON = 0.1

In [3]:
env = gym.make("FrozenLake-v0", map_name="{}x{}".format(BOARD_SIZE, BOARD_SIZE), is_slippery=True)

In [4]:
translation_dict = {
    0: ': LEFT',
    1: ': DOWN',
    2: ': RIGHT',
    3: ': UP',
}

def print_Q():
    last_key = 0
    for key, value in Q.items():
        if last_key != key[0]:
            print()
            last_key = key[0]
        key_repr = str(key[0]) + translation_dict[key[1]]
        print(key_repr, '%7.4f' % value, end=' | ')

In [5]:
def definite_move(state):
    state_action_pairs = [(state, action) for action in range(env.action_space.n)]
    max_q, best_action = Q[state_action_pairs[0]], state_action_pairs[0][1]
    for sa_pair in state_action_pairs:
        if Q[sa_pair] > max_q:
            max_q = Q[sa_pair]
            best_action = sa_pair[1]

    return best_action


def move_epsilon_greedy(state):
    rand = random.uniform(0, 1)
    if rand < EPSILON:
        return env.action_space.sample()
    else:
        return definite_move(state)

    
def move_softmax(state):
    state_action_pairs = [(state, action) for action in range(env.action_space.n)]
    probabilities = [Q[sa_pair] for sa_pair in state_action_pairs]
    normalized_probabilities = [0] * env.action_space.n
    
    for idx, p in enumerate(probabilities):
        p = p / sum(probabilities)
        normalized_probabilities[idx] = p + normalized_probabilities[idx-1]
    
    choice = random.uniform(0, 1)
    counter = 0
    while choice > normalized_probabilities[counter]:
        counter += 1
    
    return counter


def move_by_Q(state, exploration_exploitation_policy='epsilon_greedy'):
    if exploration_exploitation_policy == 'epsilon_greedy':
        return move_epsilon_greedy(state)
    elif exploration_exploitation_policy == 'softmax':
        return move_softmax(state)
    else:
        print("What")

    
def enhanced_reward(prev_state, new_state):
    if new_state == BOARD_SIZE * BOARD_SIZE - 1:
        return 50
    
    if env.desc[new_state // BOARD_SIZE][new_state % BOARD_SIZE] == b'H':
        return -10
    
    row_diff = new_state // BOARD_SIZE - prev_state // BOARD_SIZE
    col_diff = new_state % BOARD_SIZE - prev_state % BOARD_SIZE
    
    if row_diff + col_diff > 0:
        return 1
    
    return 0

In [6]:
Q = {}
def initialize_Q():
    global Q
    for state in range(env.observation_space.n):
        for action in range(env.action_space.n):
            Q[(state, action)] = random.uniform(0.49, 0.51)

def learn_Q(DECAY_RATE=0.9, LEARNING_RATE=0.1, MAX_ITER=10000, LR_SCHED=1000, LR_DECAY=0.1):
    initialize_Q()
    
    for i in range(MAX_ITER):
        state, reward, done = 0, 0, False
        env.reset()        
        if i != 0 and i % LR_SCHED == 0:
            LEARNING_RATE = LEARNING_RATE * LR_DECAY
        while not done:
            move = move_by_Q(state)

            new_state, _, done, _ = env.step(move)
            new_reward = enhanced_reward(state, new_state)
            r = new_reward - reward

            max_q_new_state = max([Q[(new_state, a)] for a in range(env.action_space.n)])
            difference = LEARNING_RATE * (r + DECAY_RATE * max_q_new_state - Q[(state, move)])
            Q[(state, move)] = Q[(state, move)] + difference

            reward = new_reward
            state = new_state

In [7]:
def run_by_Q(silent=True):
    env.reset()
    
    state, done = 0, False
    if not silent:
        env.render()
    while not done:
        state, reward, done, _ = env.step(definite_move(state))
        if not silent:
            env.render()
    
    return reward

def benchmark_Q(runs=10000):
    rewards = 0
    for i in range(runs):
        rewards += run_by_Q()

    return rewards / runs * 100

def run():
    global Q
    maximum_win_ratio = 0
    best_Q = {}
    for i in range(10):
        print("Learning...")
        learn_Q(MAX_ITER=10000, LR_SCHED=1000, LR_DECAY=0.1)
        print("Benchmarking policy...")
        win_ratio = benchmark_Q()
        print("Benchmarking done. Win Ratio: {}".format(win_ratio))
        if win_ratio > maximum_win_ratio:
            maximum_win_ratio = win_ratio
            best_Q = Q.copy()
        print()

    Q = best_Q
    print("Final win ratio: {}".format(maximum_win_ratio))

In [8]:
run()

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 39.34

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 31.330000000000002

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 34.82

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 43.730000000000004

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 46.5

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 40.28

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 46.67

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 42.84

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 38.51

Learning...
Benchmarking policy...
Benchmarking done. Win Ratio: 35.589999999999996

Final win ratio: 46.67
